In [76]:
import pandas as pd 
worldcup2018=pd.read_csv(r"C:\Projects\worldcup-social-media-nlp\Data\FIFA.csv",delimiter=',')
worldcup2022=pd.read_csv(r"C:\Projects\worldcup-social-media-nlp\Data\fifa_world_cup_2022_tweets.csv", delimiter=',')

# worldcup2018 dataset

In [ ]:
worldcup2018.columns

Index(['ID', 'lang', 'Date', 'Source', 'len', 'Orig_Tweet', 'Tweet', 'Likes',
       'RTs', 'Hashtags', 'UserMentionNames', 'UserMentionID', 'Name', 'Place',
       'Followers', 'Friends'],
      dtype='str')

In [57]:
common_ids = set(worldcup2018['ID']).intersection(set(worldcup2022['Unnamed: 0']))
print(len(common_ids))

0


**Remove unecessary columns**

In [77]:
worldcup2018=worldcup2018.drop(columns=['Followers','Friends','lang','len','ID','Name'])

In [78]:
worldcup2018.columns

Index(['Date', 'Source', 'Orig_Tweet', 'Tweet', 'Likes', 'RTs', 'Hashtags',
       'UserMentionNames', 'UserMentionID', 'Place'],
      dtype='str')

In [39]:
worldcup2018.shape

(530000, 10)

In [40]:
worldcup2018.dtypes

Date                  str
Source                str
Orig_Tweet            str
Tweet                 str
Likes               int64
RTs                 int64
Hashtags              str
UserMentionNames      str
UserMentionID         str
Place                 str
dtype: object

**Correct Data Types**

In [79]:
#change the type of the column Date from str to datetime type
worldcup2018['Date']=pd.to_datetime(worldcup2018['Date'])
worldcup2018.dtypes

Date                datetime64[us]
Source                         str
Orig_Tweet                     str
Tweet                          str
Likes                        int64
RTs                          int64
Hashtags                       str
UserMentionNames               str
UserMentionID                  str
Place                          str
dtype: object

**Fix Empty Strings**

In [80]:
# Replace empty strings, spaces, or only question marks with NaN
import numpy as np
str_cols=worldcup2018.select_dtypes(include=str).columns
worldcup2018[str_cols] = worldcup2018[str_cols].replace(r'^\s*$|^\?+$', np.nan, regex=True)

In [81]:
(worldcup2018[str_cols].isnull().sum())

Source                   6
Orig_Tweet               0
Tweet                  551
Hashtags             61543
UserMentionNames     75344
UserMentionID        74159
Place               146553
dtype: int64

In [82]:
empty_tweets = worldcup2018['Tweet'].isnull()
empty_tweets_count = worldcup2018['Tweet'].isnull().sum()
empty_hashtags_count = worldcup2018.loc[empty_tweets, 'Hashtags'].isnull().sum()
print("Number of empty Tweets is :", empty_tweets_count)
print("Number of empty hashtags when Tweet is empty:", empty_hashtags_count)


Number of empty Tweets is : 551
Number of empty hashtags when Tweet is empty: 85


In [83]:
empty_tweet_rows = worldcup2018[empty_tweets]
missing_counts = empty_tweet_rows.isnull().sum()
print("Missing counts for each column when Tweet is empty:")
print(missing_counts)

Missing counts for each column when Tweet is empty:
Date                  0
Source                1
Orig_Tweet            0
Tweet               551
Likes                 0
RTs                   0
Hashtags             85
UserMentionNames     13
UserMentionID        13
Place               159
dtype: int64


**Standardize text columns**

In [89]:
str_cols=worldcup2018.select_dtypes(include=str).columns
for col in str_cols:
    worldcup2018[col]= worldcup2018[col].str.strip().str.lower()

**Handle duplicants**

In [84]:
#chech duplicants
worldcup2018.duplicated().sum()

np.int64(4179)

In [85]:
# Show all duplicated rows
worldcup2018[worldcup2018.duplicated()]


,Date,Source,Orig_Tweet,Tweet,Likes,RTs,Hashtags,UserMentionNames,UserMentionID,Place
1575,2018-07-02 01:21:54,Twitter for iPhone,RT @MisterHeartz: GG GN and GL #WorldCup https...,GG GN and GL,0,4370,WorldCup,HRK800 Serial No. 191 919 199,MisterHeartz,NaN
1983,2018-07-02 01:18:18,Twitter for iPhone,RT @MisterHeartz: GG GN and GL #WorldCup https...,GG GN and GL,0,4370,WorldCup,HRK800 Serial No. 191 919 199,MisterHeartz,NaN
2275,2018-07-02 01:15:31,Twitter for iPhone,"RT @FIFAWorldCup: ?? | ""Japan have never made ...",Japan have never made it past the last eight s...,0,221,NaN,"FIFA World Cup ?,????????","FIFAWorldCup,jfa_samuraiblue",NaN
3114,2018-07-02 01:08:18,Twitter for iPhone,RT @brfootball: A legend says goodbye to the #...,legend says goodbye to the,0,5344,WorldCup,B/R Football,brfootball,India
3116,2018-07-02 01:08:18,Twitter for iPhone,RT @brfootball: A legend says goodbye to the #...,legend says goodbye to the,0,5344,WorldCup,B/R Football,brfootball,India
...,...,...,...,...,...,...,...,...,...,...
529531,2018-07-15 22:49:58,Twitter for Android,"RT @KhaledBeydoun: Dear France, \r\n\r\nCongra...",Dear France Congratulations on winning the of ...,0,96731,WorldCup,Khaled Beydoun,KhaledBeydoun,NaN
529588,2018-07-15 22:49:52,Twitter for iPhone,"RT @KhaledBeydoun: Dear France, \r\n\r\nCongra...",Dear France Congratulations on winning the of ...,0,96731,WorldCup,Khaled Beydoun,KhaledBeydoun,NaN
529627,2018-07-15 22:49:48,Twitter for iPhone,"RT @KhaledBeydoun: Dear France, \r\n\r\nCongra...",Dear France Congratulations on winning the of ...,0,96736,WorldCup,Khaled Beydoun,KhaledBeydoun,NaN
529860,2018-07-15 22:49:25,Twitter for Android,RT @ajplus: France just won the #WorldCup \r\n...,France just won the of its teams players are i...,0,19569,WorldCup,AJ+,ajplus,NaN


In [91]:
#Remove duplicated rows
worldcup2018 = worldcup2018.drop_duplicates()

In [92]:
worldcup2018.duplicated().sum()

np.int64(0)

In [94]:
worldcup2018.head(10)

,Date,Source,Orig_Tweet,Tweet,Likes,RTs,Hashtags,UserMentionNames,UserMentionID,Place
0,2018-07-02 01:35:45,twitter for android,rt @squawka: only two goalkeepers have saved t...,only two goalkeepers have saved three penaltie...,0,477,"worldcup,por,eng",squawka football,squawka,accra
1,2018-07-02 01:35:44,twitter for android,rt @fcbarcelona: ?? @ivanrakitic scores the wi...,scores the winning penalty to send into the qu...,0,1031,worldcup,"fc barcelona,ivan rakitic,hns | cff","fcbarcelona,ivanrakitic,hns_cff",bogor
2,2018-07-02 01:35:42,twitter for android,rt @javierfernandez: tonight we have big game....,tonight we have big game,0,488,worldcup,"javier fernandez,evgeni plushenko","javierfernandez,evgeniplushenko",NaN
3,2018-07-02 01:35:41,twitter web client,we get stronger\r\nturn the music up now\r\nwe...,we get stronger turn the music up now we got t...,0,0,"powerbyexo,worldcup,fifastadiumdj,xiuminleague","exo,fifa world cup ?","weareoneexo,fifaworldcup","zapopan, jalisco"
4,2018-07-02 01:35:40,twitter for android,rt @squawka: only two goalkeepers have saved t...,only two goalkeepers have saved three penaltie...,0,477,"worldcup,por,eng",squawka football,squawka,NaN
5,2018-07-02 01:35:40,twitter for android,"rt @fifaworldcup: ""we’re looking strong going ...",we re looking strong going into the knockout s...,0,153,bramex,"fifa world cup ?,cbf futebol,casemiro","fifaworldcup,cbf_futebol,casemiro",NaN
6,2018-07-02 01:35:40,twitter for iphone,rt @shshshshshsh555: i'm happy for #russia win...,am happy for winning especially since you know...,0,4,russia,v?deplorable?45 ??,shshshshshsh555,mount olympus
7,2018-07-02 01:35:40,twitter web client,rt @fridacarrillo05: when you see me\r\nwhen w...,when you see me when we feel the same feeling ...,0,1,"powerbyexo,worldcup,fifastadiumdj,xiuminleague",frida carrillo,fridacarrillo05,"lima, peru"
8,2018-07-02 01:35:40,twitter for android,rt @fifaworldcup: kasper schmeichel takes the ...,kasper schmeichel takes the final award of the...,0,2199,"manofthematch,croden,worldcup","fifa world cup ?,budweiser","fifaworldcup,budweiser",NaN
9,2018-07-02 01:35:40,twitter lite,rt @btsarmyna: .@bts_twt after 5 years\r\n1. g...,after years global puma ambassador lg mobile a...,0,5146,NaN,"???? & ???? ???? ™ ??,?????","btsarmyna,bts_twt",meme city


**Save a clean copy**

In [95]:
worldcup2018.to_csv(r"C:\Projects\worldcup-social-media-nlp\Data\worldcup2018_clean.csv", index=False)

# worldcup2022 dataset

In [96]:
worldcup2022.columns

Index(['Unnamed: 0', 'Date Created', 'Number of Likes', 'Source of Tweet',
       'Tweet', 'Sentiment'],
      dtype='str')

**Delete unecessary Columns**

In [97]:
worldcup2022=worldcup2022.drop(columns='Unnamed: 0')

In [105]:
#check for duplicants
worldcup2022.duplicated().sum()

np.int64(0)

In [ ]:
#check for missing values
worldcup2022.isnull().sum()

Date Created       0
Number of Likes    0
Source of Tweet    0
Tweet              0
Sentiment          0
dtype: int64

**Correct Data Types**

In [103]:
worldcup2022['Date Created']=pd.to_datetime(worldcup2022['Date Created'])
worldcup2022.dtypes

Date Created       datetime64[us, UTC]
Number of Likes                  int64
Source of Tweet                    str
Tweet                              str
Sentiment                          str
dtype: object

**Standardize text columns**

In [104]:
str_colm=worldcup2022.select_dtypes(include='str')
for col in str_colm:
    worldcup2022[col]=worldcup2022[col].str.strip().str.lower()

**Save a clean copy**

In [106]:
worldcup2022.to_csv(r"C:\Projects\worldcup-social-media-nlp\Data\worldcup2022_clean.csv")